0:  Unique identifier
1:  Last name / organization
2:  First name
3:  Middle initial
4:  Credential (example: "", "M.D.", "D.O.", "MD","DO", "OTR", "D.C.")
5:  Gender
6:  Entity ("I" for individual, "O" for organization)
7:  Address line 1
8:  Address line 2
9:  City
10: Zip
11: State
12: Country
13: Specialty ("Internal Medicine", "Pathology")
14: Participates in medicare ("Y" or "N")
15: Place of service ("F" for facility or "O" for other)
16: HCPCS code (a procedure code)
17: HCPCS code description
18: HCPCS drug indicator ("Y" or "N")
19: Line service count (people, hours, miles)
20: Beneficiary unique count (unique person, unique service)
21: Beneficiary day service count (unique person, multiple services per day)
22: Average medicare allowed amount
23: Standard deviation
24: Average submitted charges amount
25: Standard deviation
26: Average medicare payment amount
27: Standard deviation

In [14]:
# Counting procedures, building dictionaries:
from collections import Counter
code_dictionary = dict()
provider_dictionary = dict()
procedure_ctr = Counter()

infile = open("provider_utilization_2013.txt","r")
line = infile.readline()
line = infile.readline()
c = 0
max_lengths = [0 for i in range(0, 28)]

while True or c < 1000000:
    line = infile.readline()
    if(line == ""):
        break
    tokens = line.split("\t")
    code_dictionary[tokens[16]] = tokens[17]
    provider_dictionary[tokens[0]] = tokens[1:16]
    procedure_ctr[tokens[16]] += int(tokens[20])
    for i in range(0, 28):
        if len(tokens[i])> max_lengths[i]:
            max_lengths[i] = len(tokens[i])
    c+=1
infile.close()

In [ ]:
# Load the procedure descriptor table (commented for safety)
import django
import os
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "medissect.settings")
django.setup()
from explorer.models import ProcedureDescriptor
from django.db import IntegrityError
for c in code_dictionary:
    q = ProcedureDescriptor(code = c, descriptor = code_dictionary[c])
    try:
        q.save()
    except IntegrityError:
        print "Probably a double entry: " + c

In [17]:
from geopy.geocoders import GoogleV3
geolocator = GoogleV3()
c = 0
for code in provider_dictionary:
    c += 1
    if c > 100:
        break
    tokens = provider_dictionary[code]
    if len(tokens) == 15:
        address = tokens[6].title() + " " + tokens[8].title() + ", " + tokens[10]
        location = geolocator.geocode(address)
        if location:
            provider_dictionary[code] += [location.longitude, location.latitude]
        else:
            print address

In [18]:
import django
import os
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "medissect.settings")
django.setup()
from explorer.models import Provider
from django.db import IntegrityError
Provider.objects.all().delete()
for code in provider_dictionary:
    token = provider_dictionary[code]
    if len(token) == 17:
        q = Provider(
            npi = int(code),
            last_name = token[0].title(),
            first_name = token[1].title(),
            middle_initial = token[2],
            credentials = token[3],
            gender = token[4],
            is_organization = (token[5] == 'O'),
            street1 = token[6].title(),
            street2 = token[7].title(),
            city = token[8].title(),
            zipcode = token[9],
            state = token[10],
            country = token[11],
            medicare_participant = (token[13] == 'Y'),
            at_facility = (token[14] == 'F'),
            longitude = token[15],
            latitude = token[16]
        )
        try:
            q.save()
        except IntegrityError:
            print "Probably a double entry: " + code
        

In [16]:
for code in provider_dictionary:
    token = provider_dictionary[code]
    if len(token) == 17:
        print(token[15], token[16])

(-74.049097, 40.735924)
(-70.9592096, 42.0334341)
(-81.6211125, 41.5025982)
(-88.23230099999999, 43.014546)
(-92.8743008, 44.7465199)
(-117.9300468, 33.6247428)
(-90.06048299999999, 30.001112)
